In [1]:
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk

from pyspark.sql import SparkSession
from operator import add
import string
import time
import json
import re

In [2]:
spark_session = SparkSession\
    .builder\
    .master("spark://192.168.2.42:7077")\
    .appName("G18")\
    .config("spark.dynamicAllocation.enabled", True)\
    .config("spark.dynamicAllocation.shuffleTracking.enabled", True)\
    .config("spark.shuffle.service.enabled", False)\
    .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
    .config("spark.driver.port",9998)\
    .config("spark.blockManager.port",10005)\
    .getOrCreate()

# .config("spark.executor.cores",4)\

spark_context = spark_session.sparkContext
spark_context.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-03-19 15:00:12,793 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2022-03-19 15:00:14,712 WARN spark.ExecutorAllocationManager: Dynamic allocation without a shuffle service is an experimental feature.


In [3]:
_NLTK_STOPWORDS = set([re.sub('\'', '', sw) for sw in stopwords.words('english')])
_NLTK_STEMMER   = PorterStemmer()

In [4]:
def json_tuple_mapper(json_string, key, value):
    """
        Converts json string into key-value pair of ID and its associated
        comment. Comments then are lowercased.
    """
    pair = json.loads(json_string)
    id_string = pair[key]
    comment = pair[value].lower().strip()
    return (id_string, comment)

def comment_corrector_mapper(comment):
    allowed_char = string.ascii_letters + string.digits + ' '
    comment = ''.join([letter if letter in allowed_char else '' for letter in comment])
    comment = re.sub(' +', ' ', comment)
    return comment

def NLP_preprocessing_mapper(comment):
    comment_tokenized   = word_tokenize(comment)
    comment_no_stopword = [word for word in comment_tokenized if word not in _NLTK_STOPWORDS]
    comment_stemmed     = [_NLTK_STEMMER.stem(word) for word in comment_no_stopword]
    return comment_stemmed

def benchmark_rdd_evaluation(rdd):
    start = time.time()
    rdd.count()
    elapsed = time.time() - start
    print("Time taken to evaluate RDD: {} seconds".format(elapsed))

In [5]:
filename = "RC_2011-08"
lines = spark_context.textFile("hdfs://192.168.2.42:9000/user/ubuntu/data/{}".format(filename))

In [6]:
comments_rdd = lines.map(lambda json_str: json_tuple_mapper(json_str,'id','body'))\
                    .filter(lambda pair: pair[1]!='[deleted]')\
                    .mapValues(comment_corrector_mapper)\
                    .mapValues(NLP_preprocessing_mapper)

In [ ]:
benchmark_rdd_evaluation(comments_rdd)

In [ ]:
spark_context.stop()